### 손 제스처

In [10]:
import cv2
import mediapipe as mp
import numpy as np
from cvzone.Utils import overlayPNG

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# 손가락 tip index
tip_ids = [4, 8, 12, 16, 20]

# 손 흔들기 감지용
prev_x = None
wave_count = 0
wave_flag = False  # 초기화 추가!

# 이모지 이미지 로드 (투명 PNG)
emoji_images = {
    "✋": cv2.imread("emoji_1.png", cv2.IMREAD_UNCHANGED),
    "✌️": cv2.imread("emoji_3.png", cv2.IMREAD_UNCHANGED),
    "👍": cv2.imread("emoji_4.png", cv2.IMREAD_UNCHANGED),
    "👋": cv2.imread("emoji_5.png", cv2.IMREAD_UNCHANGED),
    "👌": cv2.imread("emoji_6.png", cv2.IMREAD_UNCHANGED),  
}

# 제스처 판별 함수
def detect_gesture(landmarks):
    global prev_x, wave_count, wave_flag
    fingers = []

    # 엄지 (x축 기준)
    thumb_tip = landmarks[tip_ids[0]]
    thumb_ip = landmarks[tip_ids[0] - 1]
    thumb_mcp = landmarks[2]

    if thumb_tip.x < thumb_ip.x < thumb_mcp.x or thumb_tip.x > thumb_ip.x > thumb_mcp.x:
        fingers.append(1)
    else:
        fingers.append(0)

    # 나머지 손가락 (y축 기준)
    for i in range(1, 5):
        if landmarks[tip_ids[i]].y < landmarks[tip_ids[i] - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    # 손 흔들기 감지
    x_now = landmarks[0].x
    if prev_x is not None:
        move = abs(x_now - prev_x)
        if move > 0.03:
            wave_count += 1
        else:
            wave_count = max(wave_count - 1, 0)

        if wave_count >= 3:
            wave_flag = True
    prev_x = x_now

    # OK 제스처: 엄지와 검지가 가까우면 OK
    thumb = np.array([landmarks[4].x, landmarks[4].y])
    index = np.array([landmarks[8].x, landmarks[8].y])
    dist_thumb_index = np.linalg.norm(thumb - index)

    if dist_thumb_index < 0.05:
        return "👌"

    # 제스처 판별
    if fingers == [1, 1, 1, 1, 1]:
        if wave_flag:
            wave_flag = False
            return "👋"
        else:
            return "✋"
    elif fingers == [1, 0, 0, 0, 0]:
        return "👍"
    elif fingers[1] == 1 and fingers[2] == 1 and fingers[3] == 0 and fingers[4] == 0:
        return "✌️"
    elif fingers == [1, 0, 0, 0, 0]:
        if landmarks[4].y < landmarks[2].y:
            return "👍"
        else:
            return "👊"
    else:
        return ""

# 이모지 붙이기
def draw_emoji(frame, emoji_label, size=(100, 100)):
    if emoji_label in emoji_images and emoji_images[emoji_label] is not None:
        emoji = emoji_images[emoji_label]
        emoji_resized = cv2.resize(emoji, size)
        frame = overlayPNG(frame, emoji_resized, pos=(50, 50))
    return frame

# 웹캠 시작
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    emoji = ""

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            emoji = detect_gesture(hand_landmarks.landmark)

    frame = draw_emoji(frame, emoji)
    cv2.imshow("Gesture Emoji", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

#### 가위바위보